In [1]:
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.time import BaseScheduler
from mesa.space import MultiGrid
import numpy as np
from mesa.visualization.modules import CanvasGrid
from mesa.visualization.ModularVisualization import ModularServer
from mesa.visualization.UserParam import UserSettableParameter
from mesa.visualization.modules import ChartModule, TextElement
from mesa.datacollection import DataCollector
import random
import math
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
plane_rows=[*range(1,31)]
plane_cols=['a','b','c','d','e','f']
plane_seats=[]
for row in plane_rows:
    for col in plane_cols:
        plane_seats.append((row,col))
plane_seats
actual_rows=[*range(0,30)]
actual_cols=[*range(0,7)]
actual_grid=[]
for row in actual_rows:
    for col in actual_cols:
        actual_grid.append((row,col))
actual_grid_seats=[]
for i in actual_grid:
    if i[1]!=3:
        actual_grid_seats.append(i)
seat_to_grid= dict(zip(plane_seats, actual_grid_seats))

In [3]:
# this agent is a seat          
class seat_agent(Agent):
    def __init__(self, pos, model):
        super().__init__(pos, model)
        self.occupied=0  

    def step(self):
        #print(self.dist_calc)
        return

In [4]:
# this agent is the passenger           
class passenger(Agent):
    def __init__(self, pos, model, seat):
        super().__init__(pos, model)
        self.state=1 #1 is walking, 0 stowing bags, 3 is sitting in seat
        self.seat_number = seat
        self.luggage = random.randint(1,2) #place holder to implement delay based on number of bags
        
  #function for movement along the plane (adapted from multidirectional movement function to move froward)    
    def move(self):
        neighbors = []
        x, y = self.pos
        for dx in [1]:
            for dy in [0]:
                neighbors.append((x+dx, y+dy))
        possible_steps = neighbors
        if len(self.model.grid.get_cell_list_contents([possible_steps[0]]))==0:
                new_position = self.random.choice(possible_steps)
                self.model.grid.move_agent(self, new_position)

    #Check if next to a seat   
    def check_seat(self,cell):
            for i in self.model.grid.get_cell_list_contents([cell]):
                if type(i) is seat_agent:
                    return i

   #function to check if arrived at a destination by assigned seat to adjacent seat    
    def check_arrival(self):       
        loc = []
        x, y = self.pos
        for dx in [0]:
            for dy in [1]:
                loc.append((x+dx, y+dy))       
        location=self.check_seat(loc[0])
        if seat_to_grid[self.seat_number][0]==x: 
            self.timer=0
            self.state=3

    # Sit on assigned seat    
    def sit(self,seat):
                     
        if self.state==3:
            # seat value taken from args
            self.check_seat(seat_to_grid[self.seat_number]).occupied=1
            self.model.grid.move_agent(self, seat_to_grid[self.seat_number])
            self.state=0            

    def step(self):
        if self.state==1:
            self.check_arrival()
            if self.state==1:
                self.move()
                print(self.seat_number,self.pos)
                self.check_arrival()
        elif self.state==3:
            if self.timer<10: #how long it takes to stow luggage
                self.timer+=1
            else:
                self.sit(seat_to_grid[self.seat_number])          
        return

In [5]:
#Model

class plane(Model):   
    def __init__(self,mode=1,width=30, height=7, N=180):
        super().__init__()
        
        #ticket creation
        plane_rows=[*range(1,31)]
        plane_cols=['a','b','c','d','e','f']
        plane_seats=[]
        for row in plane_rows:
            for col in plane_cols:
                plane_seats.append((row,col))
        plane_seats
        actual_rows=[*range(0,30)]
        actual_cols=[*range(0,7)]
        actual_grid=[]
        for row in actual_rows:
            for col in actual_cols:
                actual_grid.append((row,col))
        actual_grid_seats=[]
        for i in actual_grid:
            if i[1]!=3:
                actual_grid_seats.append(i)
        seat_to_grid = dict(zip(plane_seats, actual_grid_seats))
  
        ####BACK TO FRONT LOADING
        def flatten(plane_seats_sector):   
            flat_list = []
            for sublist in plane_seats_sector:
                for item in sublist:
                    flat_list.append(item)
            return flat_list   
        all_sectors=[]
        plane_seats_sector=[]
        p=0
        v=30
        for i in range(6):
            plane_seats_sector=[]
            plane_seats_sector.append(plane_seats[p:v])
            flp=flatten(plane_seats_sector)
            random.shuffle(flp)

            all_sectors.append(flp)
            p+=30
            v+=30
        all_sectors
        btf=flatten(all_sectors)
        
        
        ### Model Params
        self.mode=mode
        self.width=width
        self.height=height
        self.grid = MultiGrid(width, height, False)
        self.place=0
        self.schedule = BaseScheduler(self)
        self.all_passengers=[]
        #depending on selected mode, tickets are allocated 
        if self.mode==1:
            self.tickets=plane_seats
        elif self.mode==2:
            self.tickets=btf
        self.builder=[]
        type_distribution = np.genfromtxt("plane.txt")
        self.counter=0.01

        #create seat
        for _,x, y in self.grid.coord_iter():
            if type_distribution[x,y]==1: #1 is the chosen identifier for a seat in the txt doc
                new_seat=seat_agent((x,y),self)
                self.grid.place_agent(new_seat,(x, y))
                self.schedule.add(new_seat)

        #create passengers 
        for i in range(180):      
            if True:
                a = passenger(i, self, self.tickets[-1])
                self.tickets.pop()
                self.all_passengers.append(a)
                if self.mode==1:
                    random.shuffle(self.all_passengers)
        self.running = True     
   
    #what happens every step
    def step(self):
        self.counter+=1
        self.schedule.step()
        if len(self.all_passengers)>0 and len(self.grid.get_cell_list_contents([(0,3)]))==0:
                self.grid.place_agent(self.all_passengers[0],(0,3))
                self.schedule.add(self.all_passengers[0])
                self.all_passengers.pop(0)

        self.place+=1      

In [6]:
def agent_portrayal(agent):
    if agent is None:
        return 
    
    

    portrayal = {}

    if type(agent) is passenger:
        portrayal = {"Shape": "circle", "Filled": "true", "r": 0.5, "Color" : "#32a883", "Layer":5,}
    elif type(agent) is seat_agent: 
       
        portrayal = {"Shape": "rect","Filled": "true","Layer": 3 , "w":1 , "h":1,}
        
        if agent.occupied== 0:
            portrayal["Shape"] = "seat_a.png"
        
        else:
            portrayal["Shape"] = "seat_o.png" 
    
    return portrayal


grid = CanvasGrid(agent_portrayal, 30, 7, 690,161)


In [7]:




port_num=8006
server = ModularServer(plane,
                       [grid], 
                       "Plane Model")
server.port = port_num
server.launch()  

Interface starting at http://127.0.0.1:8006


RuntimeError: This event loop is already running

Socket opened!
{"type":"reset"}
